# INTRODUCTION

The purpose of this demo is to illustrate the setup and interaction of a distributed system using Kodo services. The demo showcases the following key steps:

1. registry startup
2. node register
3. registry peer connect
4. node restarts and updates

We create a _multiprocessing_ runtime environment for seamless communication over http sockets.

In [44]:
from multiprocessing import Process
from kodo.service.node import run_service
import httpx

# REGISTRY1 STARTUP

We startup a `registry1` and a `node1` connects to this `registry1`.

Next we startup another `registry2` and a `node2` connects to this`registry2`.

In [45]:
registry1ip = "http://localhost:3366"
node1ip = "http://localhost:3367"
registry2ip = "http://localhost:3368"
node2ip = "http://localhost:3369"

In [46]:
registry1kw = {
    "url": registry1ip, 
    "organization": "registry1",
    "node": False, 
    "registry": True,
    # initialise the cache, which we will use later
    "cache": "./registry1.json",
    # start from scratch
    "reset": True
}
registry1 = Process(target=run_service, kwargs=registry1kw)
registry1.start()

## HANDSHAKE

The registry is up-and-running.

In [47]:
resp = httpx.get(registry1ip)
resp.json()

INFO - 2024-12-26 11:04:33,409 - kodo - node - http://localhost:3366: registry startup
INFO - 2024-12-26 11:04:33,409 - kodo - node - http://localhost:3366: ignore cache ./registry1.json
INFO - 2024-12-26 11:04:33,409 - kodo - node - http://localhost:3366: successfully started registry
INFO - 2024-12-26 11:04:33,409 - uvicorn.error - server - Started server process [56353]
INFO - 2024-12-26 11:04:33,409 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 11:04:33,414 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 11:04:33,469 - uvicorn.access - httptools_impl - 127.0.0.1:51568 - "GET / HTTP/1.1" 200


{'url': 'http://localhost:3366',
 'organization': 'registry1',
 'connection': {},
 'node': False,
 'registry': True,
 'provider': False,
 'started_at': '2024-12-26T11:04:33.414447',
 'idle': True,
 'now': '2024-12-26T11:04:33.469263',
 'message': []}

This registry `_cache` is empty. 

In [48]:
resp = httpx.get(f"{registry1ip}/_cache")
resp.json()

INFO - 2024-12-26 11:04:43,567 - uvicorn.access - httptools_impl - 127.0.0.1:51576 - "GET /_cache HTTP/1.1" 200


{'url': 'http://localhost:3366',
 'organization': 'registry1',
 'feed': True,
 'nodes': {},
 'providers': {}}

The registry `count` is empty.

In [49]:
resp = httpx.get(f"{registry1ip}/counts")
resp.json()

INFO - 2024-12-26 11:04:44,580 - uvicorn.access - httptools_impl - 127.0.0.1:51578 - "GET /counts HTTP/1.1" 200


{'total': 0, 'node': {}, 'organization': {}, 'tags': {}}

INFO - 2024-12-26 11:05:03,183 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:04,152 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:04,545 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:04,834 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:05,073 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:05,302 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:05,530 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:05,757 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1.1" 200
INFO - 2024-12-26 11:05:05,970 - uvicorn.access - httptools_impl - 127.0.0.1:51592 - "GET /schema HTTP/1

# node1 registers with registry1

Start `node1` with a flow `mytest`. The loader must either be a factor string (e.g. `module.path:func`) or a _Callable_.
In this example we use a callable `my_loader` in `myflow`.

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
```

In [7]:
import importlib
import flows
importlib.reload(flows)

<module 'flows' from '/Users/raum/Project/kodo-core/tests/flows.py'>

```json
{
  "url": "string",
  "organization": {},
  "flows": {
    "uid: string": {
      "url": "string",
      "name": "string",
      "description": "string",
      "author": {},
      "tags": []
    },
    "uid: string": {
      "url": "string",
      "name": "string",
      "description": {},
      "author": {},
      "tags": []
    }
  }
}
```

* client: node1
* method: `POST /register`
* server: `http://registry1`
* required parameters
  * url (str)
  * flows (dict)
  * 
```json
{
    "url": str,
    "organization": str,
    "flows": {
        "{url: str}": {
            "url": str,
            "name": str,
            "description": str,
            "tags": str,
            "author": str
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry1 in response delivers:

```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:03:02.245504",
    "idle": true,
    "now": "2024-12-25T18:03:12.990684",
    "message": [
        "registered node http://localhost:3367 (first visit)"
    ]
}
```

In [8]:
node1kw = {
    "url": node1ip, 
    "connect": [registry1ip],
    "organization": "node1",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader1
}
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

INFO - 2024-12-26 10:28:59,606 - kodo - node - http://localhost:3367: node startup
INFO - 2024-12-26 10:28:59,606 - kodo - node - http://localhost:3367: loaded 1 flows
INFO - 2024-12-26 10:28:59,606 - kodo - node - http://localhost:3367: loaded 1 entries
INFO - 2024-12-26 10:28:59,606 - kodo - node - http://localhost:3367: successfully started node
INFO - 2024-12-26 10:28:59,607 - uvicorn.error - server - Started server process [56180]
INFO - 2024-12-26 10:28:59,607 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:28:59,609 - kodo - routes - http://localhost:3367: semaphore +1 to 1
INFO - 2024-12-26 10:28:59,609 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 10:28:59,609 - kodo - routes - http://localhost:3367 establishes connection (http://localhost:3366/register)
INFO - 2024-12-26 10:28:59,624 - uvicorn.error - server - Uvicorn running on http://localhost:3367 (Press CTRL+C to quit)
INFO - 2024-12-26 10:28:59,626 - kodo - routes - h

The node is up-and-running and connected to http://localhost:3366 (registry1)

In [9]:
resp = httpx.get(node1ip)
resp.json()

INFO - 2024-12-26 10:29:03,112 - uvicorn.access - httptools_impl - ::1:51149 - "GET / HTTP/1.1" 200


{'url': 'http://localhost:3367',
 'organization': 'node1',
 'connection': {'http://localhost:3366': '2024-12-26T10:28:59.627614'},
 'node': True,
 'registry': False,
 'provider': False,
 'started_at': '2024-12-26T10:28:59.609470',
 'idle': True,
 'now': '2024-12-26T10:29:03.111910',
 'message': []}

There is no such thing as a node `_cache`.

In [10]:
resp = httpx.get(f"{node1ip}/_cache")
assert resp.status_code == 404

INFO - 2024-12-26 10:29:04,225 - uvicorn.access - httptools_impl - ::1:51151 - "GET /_cache HTTP/1.1" 404


But a flow has been created!

In [11]:
resp = httpx.get(f"{node1ip}/flows")
resp.json()

INFO - 2024-12-26 10:29:05,683 - uvicorn.access - httptools_impl - ::1:51154 - "GET /flows HTTP/1.1" 200


{'total': 1,
 'filtered': 1,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3367',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': None,
   'modified': None,
   'heartbeat': None,
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

Please note that the node itself does not report on any timestamps. This feature is reserved to registries and providers.

The flow is reporting it's `counts`.

In [12]:
resp = httpx.get(f"{node1ip}/counts")
resp.json()

INFO - 2024-12-26 10:29:07,542 - uvicorn.access - httptools_impl - ::1:51159 - "GET /counts HTTP/1.1" 200


{'total': 1,
 'node': {'http://localhost:3367': 1},
 'organization': {'node1': 1},
 'tags': {}}

# Visit the registry

The registry reports timestamps. This is in contrast to the node.

In [13]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

INFO - 2024-12-26 10:29:10,288 - uvicorn.access - httptools_impl - 127.0.0.1:51167 - "GET /flows HTTP/1.1" 200


{'total': 1,
 'filtered': 1,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3366',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': '2024-12-26T10:28:59.626308',
   'modified': '2024-12-26T10:28:59.626308',
   'heartbeat': '2024-12-26T10:28:59.626308',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

## node1 restart

On restart of the node the registry updates the _modified_ timestamp.

In [14]:
node1.terminate()

INFO - 2024-12-26 10:29:14,580 - uvicorn.error - server - Shutting down
INFO - 2024-12-26 10:29:14,682 - uvicorn.error - on - Waiting for application shutdown.
INFO - 2024-12-26 10:29:14,683 - kodo - routes - http://localhost:3367 shutdown now
INFO - 2024-12-26 10:29:14,683 - uvicorn.error - on - Application shutdown complete.


again on `/register` the node sends: 
```json
{
    "url": "http://localhost:3367",
    "organization": "node1",
    "flows": {
        "/mytest": {
            "url": "/mytest",
            "name": "mytest",
            "description": null,
            "tags": null,
            "author": null
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

and the registry returns: 
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:03:02.245504",
    "idle": true,
    "now": "2024-12-25T18:05:13.972890",
    "message": [
        "registered node http://localhost:3367 (seen previously 2024-12-25 18:03:12.989989)"
    ]
}```

In [15]:
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

INFO - 2024-12-26 10:29:20,643 - kodo - node - http://localhost:3367: node startup
INFO - 2024-12-26 10:29:20,644 - kodo - node - http://localhost:3367: loaded 1 flows
INFO - 2024-12-26 10:29:20,644 - kodo - node - http://localhost:3367: loaded 1 entries
INFO - 2024-12-26 10:29:20,644 - kodo - node - http://localhost:3367: successfully started node
INFO - 2024-12-26 10:29:20,644 - uvicorn.error - server - Started server process [56195]
INFO - 2024-12-26 10:29:20,644 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:29:20,646 - kodo - routes - http://localhost:3367: semaphore +1 to 1
INFO - 2024-12-26 10:29:20,646 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 10:29:20,647 - kodo - routes - http://localhost:3367 establishes connection (http://localhost:3366/register)
INFO - 2024-12-26 10:29:20,663 - uvicorn.error - server - Uvicorn running on http://localhost:3367 (Press CTRL+C to quit)
INFO - 2024-12-26 10:29:20,664 - kodo - routes - h

In [16]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

INFO - 2024-12-26 10:29:22,343 - uvicorn.access - httptools_impl - 127.0.0.1:51174 - "GET /flows HTTP/1.1" 200


{'total': 1,
 'filtered': 1,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3366',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': '2024-12-26T10:28:59.626308',
   'modified': '2024-12-26T10:29:20.664197',
   'heartbeat': '2024-12-26T10:29:20.664197',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

On registry restart the flow collection is retrieved from cache. Both the _created_ and _modified_ timestamp remain unchanged.

In [17]:
registry1.terminate()

INFO - 2024-12-26 10:29:23,889 - uvicorn.error - server - Shutting down
INFO - 2024-12-26 10:29:23,991 - uvicorn.error - on - Waiting for application shutdown.
INFO - 2024-12-26 10:29:23,992 - kodo - routes - http://localhost:3366 shutdown now


In [18]:
# enable cache!
registry1kw["reset"] = False
registry1 = Process(target=run_service, kwargs=registry1kw)
registry1.start()

INFO - 2024-12-26 10:29:26,052 - kodo - node - http://localhost:3366: registry startup
INFO - 2024-12-26 10:29:26,052 - kodo - node - http://localhost:3366: loading from cache ./registry1.json
INFO - 2024-12-26 10:29:26,052 - kodo - node - http://localhost:3366: loaded 1 own nodes and 0 nodes from 0 providers
INFO - 2024-12-26 10:29:26,052 - kodo - node - http://localhost:3366: set organization = 'registry1'
INFO - 2024-12-26 10:29:26,052 - kodo - node - http://localhost:3366: successfully started registry
INFO - 2024-12-26 10:29:26,052 - uvicorn.error - server - Started server process [56199]
INFO - 2024-12-26 10:29:26,053 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:29:26,055 - uvicorn.error - on - Application startup complete.


In [19]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

INFO - 2024-12-26 10:29:28,865 - uvicorn.access - httptools_impl - 127.0.0.1:51177 - "GET /flows HTTP/1.1" 200


{'total': 1,
 'filtered': 1,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3366',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': '2024-12-26T10:28:59.626308',
   'modified': '2024-12-26T10:29:20.664197',
   'heartbeat': '2024-12-26T10:29:20.664197',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

# REGISTRY2 STARTUP

Create `registry2` and connect with `registry1`.

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
    registry2 --> registry1
```

`registry2` at startup connects to `registry1` and POST /connect: 
```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "feed": true,
    "nodes": {}
}
```

`registry1` returns
```json
{
    "providers": {
        "url": "http://localhost:3366",
        "organization": "registry1",
        "feed": true,
        "nodes": {
            "http://localhost:3367": {
                "url": "http://localhost:3367",
                "organization": "node1",
                "flows": {
                    "/mytest": {
                        "url": "/mytest",
                        "name": "mytest",
                        "description": null,
                        "tags": null,
                        "author": null
                    }
                },
                "created": "2024-12-25T18:03:12.989989",
                "modified": "2024-12-25T18:05:13.972356",
                "heartbeat": "2024-12-25T18:05:13.972356",
                "status": "unknown"
            }
        }
    },
    "message": [
        "registry http://localhost:3368 connected (first visit)"
    ]
}
```

In [20]:
registry2kw = {
    "url": registry2ip, 
    "connect": [registry1ip],
    "organization": "registry2",
    "node": False, 
    "registry": True,
    # initialise the cache, which we will use later
    "cache": "./registry2.json",
    # start from scratch
    "reset": True
}
registry2 = Process(target=run_service, kwargs=registry2kw)
registry2.start()

INFO - 2024-12-26 10:29:35,743 - kodo - node - http://localhost:3368: registry startup
INFO - 2024-12-26 10:29:35,743 - kodo - node - http://localhost:3368: ignore cache ./registry2.json
INFO - 2024-12-26 10:29:35,743 - kodo - node - http://localhost:3368: successfully started registry
INFO - 2024-12-26 10:29:35,743 - uvicorn.error - server - Started server process [56208]
INFO - 2024-12-26 10:29:35,744 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:29:35,746 - kodo - routes - http://localhost:3368: semaphore +1 to 1
INFO - 2024-12-26 10:29:35,746 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 10:29:35,746 - kodo - routes - http://localhost:3368 establishes connection (http://localhost:3366/connect)
INFO - 2024-12-26 10:29:35,765 - kodo - routes - http://localhost:3366 on /connect
INFO - 2024-12-26 10:29:35,765 - kodo - routes - http://localhost:3366: provider dump to ./registry1.json
INFO - 2024-12-26 10:29:35,765 - kodo - routes -

# HANDSHAKE

The registry is up-and-running and connected to `registry1`.

In [21]:
resp = httpx.get(registry2ip)
resp.json()

INFO - 2024-12-26 10:29:40,169 - uvicorn.access - httptools_impl - 127.0.0.1:51181 - "GET / HTTP/1.1" 200


{'url': 'http://localhost:3368',
 'organization': 'registry2',
 'connection': {'http://localhost:3366': '2024-12-26T10:29:35.766423'},
 'node': False,
 'registry': True,
 'provider': False,
 'started_at': '2024-12-26T10:29:35.746013',
 'idle': True,
 'now': '2024-12-26T10:29:40.169506',
 'message': []}

Both registries `registry1` and `registry2` are in-sync.

In [22]:
resp1 = httpx.get(f"{registry1ip}/flows")
resp2 = httpx.get(f"{registry2ip}/flows")

INFO - 2024-12-26 10:29:41,476 - uvicorn.access - httptools_impl - 127.0.0.1:51183 - "GET /flows HTTP/1.1" 200
INFO - 2024-12-26 10:29:41,505 - uvicorn.access - httptools_impl - 127.0.0.1:51185 - "GET /flows HTTP/1.1" 200


In [23]:
item1 = resp1.json()["items"][0]
item2 = resp2.json()["items"][0]

# remove the registry masquerade
item1.pop("registry")
item2.pop("registry")

assert item1 == item2

# Node2 connects to registry2
## Node2 joins the mesh

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
    registry2 --> registry1
    node2 --> registry2
    /mytest2 --> node2
```

node2 `POST http://registry2/register` with

```json
{
    "url": "http://localhost:3369",
    "organization": "node2",
    "flows": {
        "/mytest2": {
            "url": "/mytest2",
            "name": "mytest2",
            "description": null,
            "tags": null,
            "author": null
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry2 responds with

```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {
        "http://localhost:3366": "2024-12-25T18:07:39.033999"
    },
    "started_at": "2024-12-25T18:07:39.013209",
    "idle": false,
    "now": "2024-12-25T18:35:36.659965",
    "message": [
        "registered node http://localhost:3369 (first visit)",
        "feed forward node to http://localhost:3366"
    ]
}
```

registry2 feed forwards the node update to registry1 with `POST http://registry1/update/node`

```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "feed": false,
    "nodes": {
        "http://localhost:3369": {
            "url": "http://localhost:3369",
            "organization": "node2",
            "flows": {
                "/mytest2": {
                    "url": "/mytest2",
                    "name": "mytest2",
                    "description": null,
                    "tags": null,
                    "author": null
                }
            },
            "created": "2024-12-25T18:35:36.651614",
            "modified": "2024-12-25T18:35:36.651614",
            "heartbeat": "2024-12-25T18:35:36.651614",
            "status": "unknown"
        }
    }
}
```

response of `registry1` is

```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:07:18.806234",
    "idle": true,
    "now": "2024-12-25T18:35:36.693275",
    "message": [
        "created node http://localhost:3368"
    ]
}
```

In [24]:
importlib.reload(flows)

<module 'flows' from '/Users/raum/Project/kodo-core/tests/flows.py'>

In [25]:
node2kw = {
    "url": node2ip, 
    "connect": [registry2ip],
    "organization": "node2",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader2
}
node2 = Process(target=run_service, kwargs=node2kw)
node2.start()

INFO - 2024-12-26 10:29:55,079 - kodo - node - http://localhost:3369: node startup
INFO - 2024-12-26 10:29:55,079 - kodo - node - http://localhost:3369: loaded 1 flows
INFO - 2024-12-26 10:29:55,079 - kodo - node - http://localhost:3369: loaded 1 entries
INFO - 2024-12-26 10:29:55,079 - kodo - node - http://localhost:3369: successfully started node
INFO - 2024-12-26 10:29:55,079 - uvicorn.error - server - Started server process [56222]
INFO - 2024-12-26 10:29:55,079 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:29:55,082 - kodo - routes - http://localhost:3369: semaphore +1 to 1
INFO - 2024-12-26 10:29:55,082 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 10:29:55,082 - kodo - routes - http://localhost:3369 establishes connection (http://localhost:3368/register)
INFO - 2024-12-26 10:29:55,097 - uvicorn.error - server - Uvicorn running on http://localhost:3369 (Press CTRL+C to quit)
INFO - 2024-12-26 10:29:55,099 - kodo - routes - h

The node is up-and-running and connected to http://localhost:3368 (registry2)

In [26]:
resp = httpx.get(node2ip)
resp.json()

INFO - 2024-12-26 10:30:00,622 - uvicorn.access - httptools_impl - ::1:51195 - "GET / HTTP/1.1" 200


{'url': 'http://localhost:3369',
 'organization': 'node2',
 'connection': {'http://localhost:3368': '2024-12-26T10:29:55.100121'},
 'node': True,
 'registry': False,
 'provider': False,
 'started_at': '2024-12-26T10:29:55.082250',
 'idle': True,
 'now': '2024-12-26T10:30:00.622118',
 'message': []}

The flow has been created!

In [27]:
resp = httpx.get(f"{node2ip}/flows")
resp.json()

INFO - 2024-12-26 10:30:04,630 - uvicorn.access - httptools_impl - ::1:51196 - "GET /flows HTTP/1.1" 200


{'total': 1,
 'filtered': 1,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3369',
   'node': 'http://localhost:3369',
   'organization': 'node2',
   'created': None,
   'modified': None,
   'heartbeat': None,
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

Again the node does not report on any timestamps.

# Visit all registries

In [28]:
resp1 = httpx.get(f"{registry1ip}/flows")
resp2 = httpx.get(f"{registry2ip}/flows")

INFO - 2024-12-26 10:30:05,972 - uvicorn.access - httptools_impl - 127.0.0.1:51198 - "GET /flows HTTP/1.1" 200
INFO - 2024-12-26 10:30:05,997 - uvicorn.access - httptools_impl - 127.0.0.1:51200 - "GET /flows HTTP/1.1" 200


In [29]:
resp1.json()

{'total': 2,
 'filtered': 2,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3366',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': '2024-12-26T10:28:59.626308',
   'modified': '2024-12-26T10:29:20.664197',
   'heartbeat': '2024-12-26T10:29:20.664197',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []},
  {'registry': 'http://localhost:3366',
   'node': 'http://localhost:3369',
   'organization': 'node2',
   'created': '2024-12-26T10:29:55.098981',
   'modified': '2024-12-26T10:29:55.098981',
   'heartbeat': '2024-12-26T10:29:55.098981',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

In [30]:
resp2.json()

{'total': 2,
 'filtered': 2,
 'p': 0,
 'pp': 10,
 'items': [{'registry': 'http://localhost:3368',
   'node': 'http://localhost:3367',
   'organization': 'node1',
   'created': '2024-12-26T10:28:59.626308',
   'modified': '2024-12-26T10:29:20.664197',
   'heartbeat': '2024-12-26T10:29:20.664197',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []},
  {'registry': 'http://localhost:3368',
   'node': 'http://localhost:3369',
   'organization': 'node2',
   'created': '2024-12-26T10:29:55.098981',
   'modified': '2024-12-26T10:29:55.098981',
   'heartbeat': '2024-12-26T10:29:55.098981',
   'url': '/test/hymn1',
   'name': 'Hymn Creator',
   'author': 'missing author',
   'description': 'missing description',
   'tags': []}],
 'by': 'name, node, url',
 'q': None}

In [31]:
from IPython.display import display, HTML

In [32]:
resp1 = httpx.get(f"{registry1ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

INFO - 2024-12-26 10:30:09,669 - uvicorn.access - httptools_impl - 127.0.0.1:51202 - "GET /flows?format=html HTTP/1.1" 200


,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags
0,http://localhost:3366,http://localhost:3367,node1,2024-12-26 10:28:59.626308,2024-12-26 10:29:20.664197,2024-12-26 10:29:20.664197,/test/hymn1,Hymn Creator,missing author,missing description,[]
1,http://localhost:3366,http://localhost:3369,node2,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,/test/hymn1,Hymn Creator,missing author,missing description,[]


In [33]:
resp1 = httpx.get(f"{registry2ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

INFO - 2024-12-26 10:30:10,518 - uvicorn.access - httptools_impl - 127.0.0.1:51204 - "GET /flows?format=html HTTP/1.1" 200


,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags
0,http://localhost:3368,http://localhost:3367,node1,2024-12-26 10:28:59.626308,2024-12-26 10:29:20.664197,2024-12-26 10:29:20.664197,/test/hymn1,Hymn Creator,missing author,missing description,[]
1,http://localhost:3368,http://localhost:3369,node2,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,/test/hymn1,Hymn Creator,missing author,missing description,[]


# Node updates

When a node reconnects, the news is broadcasted from the sourcing registry to all peers.

In [35]:
node1.terminate()

INFO - 2024-12-26 10:30:18,952 - uvicorn.error - server - Shutting down
INFO - 2024-12-26 10:30:19,054 - uvicorn.error - on - Waiting for application shutdown.
INFO - 2024-12-26 10:30:19,055 - kodo - routes - http://localhost:3367 shutdown now
INFO - 2024-12-26 10:30:19,055 - uvicorn.error - on - Application shutdown complete.


In [36]:
node1kw = {
    "url": node1ip, 
    "connect": [registry1ip],
    "organization": "node1",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader3
}

node1 connects to registry1
```json
{
    "url": "http://localhost:3367",
    "organization": "node1",
    "flows": {
        "/mytest2": {
            "url": "/mytest2",
            "name": "mytest2",
            "description": "This is a testing flow",
            "tags": null,
            "author": "michi.rau@gmail.com"
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry1 responds with
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:07:18.806234",
    "idle": false,
    "now": "2024-12-25T18:44:02.640826",
    "message": [
        "registered node http://localhost:3367 (seen previously 2024-12-25 18:03:12.989989)",
        "feed forward node to http://localhost:3368"
    ]
}
```

registry1 broadcasts the update of node1 to registry2
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "feed": false,
    "nodes": {
        "http://localhost:3367": {
            "url": "http://localhost:3367",
            "organization": "node1",
            "flows": {
                "/mytest2": {
                    "url": "/mytest2",
                    "name": "mytest2",
                    "description": "This is a testing flow",
                    "tags": null,
                    "author": "michi.rau@gmail.com"
                }
            },
            "created": "2024-12-25T18:03:12.989989",
            "modified": "2024-12-25T18:44:02.640011",
            "heartbeat": "2024-12-25T18:44:02.640011",
            "status": "unknown"
        }
    }
}
```

registry2 responds with
```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {
        "http://localhost:3366": "2024-12-25T18:35:36.694219"
    },
    "started_at": "2024-12-25T18:07:39.013209",
    "idle": true,
    "now": "2024-12-25T18:44:02.660559",
    "message": [
        "updated node http://localhost:3366"
    ]
}
```

In [37]:
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

INFO - 2024-12-26 10:30:38,432 - kodo - node - http://localhost:3367: node startup
INFO - 2024-12-26 10:30:38,432 - kodo - node - http://localhost:3367: loaded 1 flows
INFO - 2024-12-26 10:30:38,432 - kodo - node - http://localhost:3367: loaded 1 entries
INFO - 2024-12-26 10:30:38,432 - kodo - node - http://localhost:3367: successfully started node
INFO - 2024-12-26 10:30:38,432 - uvicorn.error - server - Started server process [56241]
INFO - 2024-12-26 10:30:38,432 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 10:30:38,434 - kodo - routes - http://localhost:3367: semaphore +1 to 1
INFO - 2024-12-26 10:30:38,434 - uvicorn.error - on - Application startup complete.
INFO - 2024-12-26 10:30:38,435 - kodo - routes - http://localhost:3367 establishes connection (http://localhost:3366/register)
INFO - 2024-12-26 10:30:38,450 - uvicorn.error - server - Uvicorn running on http://localhost:3367 (Press CTRL+C to quit)
INFO - 2024-12-26 10:30:38,451 - kodo - routes - h

## revisit registries

In [38]:
resp1 = httpx.get(f"{registry1ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

INFO - 2024-12-26 10:30:44,915 - uvicorn.access - httptools_impl - 127.0.0.1:51213 - "GET /flows?format=html HTTP/1.1" 200


,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags
0,http://localhost:3366,http://localhost:3367,node1,2024-12-26 10:28:59.626308,2024-12-26 10:30:38.451698,2024-12-26 10:30:38.451698,/test/hymn1,Hymn Creator,michi.rau@gmail.com,This is a testing flow,[]
1,http://localhost:3366,http://localhost:3369,node2,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,/test/hymn1,Hymn Creator,missing author,missing description,[]


In [39]:
resp2 = httpx.get(f"{registry2ip}/flows?format=html")
display(HTML(resp2.content.decode("utf-8")))

INFO - 2024-12-26 10:30:45,624 - uvicorn.access - httptools_impl - 127.0.0.1:51215 - "GET /flows?format=html HTTP/1.1" 200


,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags
0,http://localhost:3368,http://localhost:3367,node1,2024-12-26 10:28:59.626308,2024-12-26 10:30:38.451698,2024-12-26 10:30:38.451698,/test/hymn1,Hymn Creator,michi.rau@gmail.com,This is a testing flow,[]
1,http://localhost:3368,http://localhost:3369,node2,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,2024-12-26 10:29:55.098981,/test/hymn1,Hymn Creator,missing author,missing description,[]


# TERMINATION

In [50]:
registry1.terminate()

INFO - 2024-12-26 11:22:04,734 - uvicorn.error - server - Shutting down
INFO - 2024-12-26 11:22:04,835 - uvicorn.error - on - Waiting for application shutdown.
INFO - 2024-12-26 11:22:04,836 - kodo - routes - http://localhost:3366 shutdown now


In [51]:
node1.terminate()

In [52]:
registry2.terminate()

In [53]:
node2.terminate()